In [4]:
import time
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
import shap

X, y = shap.datasets.diabetes()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

X_test = X_test.reset_index(drop=True)
X_train_summary = shap.kmeans(X_train, 10)

nn = MLPRegressor(solver="lbfgs", alpha=1e-1, hidden_layer_sizes=(5, 2), random_state=0)
nn.fit(X_train, y_train)

def print_accuracy(f):
    print(
        f"Root mean squared test error = {np.sqrt(np.mean((f(X_test) - y_test) ** 2))}"
    )
    time.sleep(0.5)  # to let the print get out before any progress bars

print_accuracy(nn.predict)

H:\feature\.venv11\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Root mean squared test error = 58.13233529996922


In [5]:
def my_predict(X):
    return nn.predict(X)

In [26]:
# explain all the predictions in the test set
explainer = shap.KernelExplainer(my_predict, X_train_summary)
shap_values = explainer(X_test)
shap.plots.bar(shap_values[0])

H:\feature\.venv11\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(


  0%|          | 0/89 [00:00<?, ?it/s]

H:\feature\.venv11\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(
H:\feature\.venv11\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(
H:\feature\.venv11\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(
H:\feature\.venv11\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(
H:\feature\.venv11\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(
H:\feature\.venv11\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with

KeyboardInterrupt: 

In [33]:
import pandas as pd

# do the same changes to multiple data frames
def combine_columns(df, columns):
    #combine column names to a string
    combined_name = ', '.join(columns)
    #combine the columns
    df[combined_name] = df.apply(lambda x: tuple(x[columns]), axis=1)
    #delete the original columns
    df = df.drop(columns, axis=1)
    
    return df

def split_columns(df, combined_name):
    #get the original names
    columns = combined_name.split(', ')
    #split the combined column
    df = df.join(df[combined_name].apply(pd.Series))
    #delete the combined column
    df = df.drop(combined_name, axis=1)
    #rename the newly created columns
    df.columns = df.columns[:len(df.columns)-len(columns)].tolist() + columns
    return df

def split_all_columns(df):
    #split all combined columns
    for column in df.columns:
        if type(df[column][0]) == tuple:
            df = split_columns(df, column)
    return df

,c,a,b
0,7,1,4
1,8,2,5
2,9,3,6


In [13]:
COLUMNS = X_train.columns
def detangled_predict(X):
    return nn.predict(split_all_columns(X).reindex(columns=COLUMNS))

In [9]:
#convert x_train_summary from DenseData to DataFrame
X_train_means = X_train.mean().to_frame().T


In [34]:
#entangle s1 and s2
tmp_X_test = X_test.copy()
tmp_X_test = combine_columns(tmp_X_test, ['s1', 's2'])
tmp_X_train_means = X_train_means.copy()
tmp_X_train_means = combine_columns(tmp_X_train_means, ['s1', 's2'])

explainer = shap.KernelExplainer(detangled_predict, tmp_X_train_means, COLUMNS, keep_index=True)
shap_values = explainer(tmp_X_test)
shap.plots.bar(shap_values[0])

  0%|          | 0/89 [00:00<?, ?it/s]

ValueError: Input X contains NaN.
MLPRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [25]:
tmp_X_test = X_test.copy()
tmp_X_test = combine_columns(tmp_X_test, ['s1', 's2'])
detangled_predict(tmp_X_test)


array([244.17416526, 253.77566168, 161.32408257, 105.46732634,
       179.98879276, 265.46111709, 104.33040162, 196.74915838,
       122.21402313, 243.62346552, 179.19718382, 181.60493132,
       100.8952043 ,  81.82360214, 253.74568857,  81.82360214,
       149.48239406,  81.82360214,  81.82360214, 216.64045132,
       188.96497734, 164.39666849, 169.25788311, 142.44414557,
       200.17475791, 171.92049982, 121.90067804,  81.82360214,
       199.72500377, 161.82726914, 182.50649032,  81.82360214,
       128.14598366, 137.54932068, 142.7581079 , 200.11799424,
       163.81336882, 196.92044251, 111.75083038, 211.11331844,
        81.82360214, 161.11109051, 147.23231725, 192.96398456,
       171.79475489,  81.82360214, 150.37029369, 121.69564147,
       118.50567395, 237.29377013, 165.74908775,  81.82360214,
       133.90325241, 163.22203313, 244.05605526, 175.69825647,
       192.54972537,  99.98135816, 128.15624762, 175.05439353,
       219.04974849, 167.9390542 , 140.50881664, 101.69